# Extract Protokolle

In [2]:
from docx import *

In [3]:
document = Document('../export/Files/2021-09-27-17fa216f98fd4d38afd5afb53743ec65-332.docx')

In [12]:
for para in document.paragraphs:
    for run in para.runs:
        if run.italic :
            index = para.text.index(run.text)
            #print(index)
            if index != 0:
                print("xx[%s]xx (%s)" % (run.text, index))


            #print(para.text)

            break

xx[Benno Scherrer (GLP, Uster)]xx (9)
xx[(schriftliches Verfahren)]xx (13)
xx[Ratspräsident Benno Scherrer:]xx (35)
xx[(Die Anwesenden erhe]xx (447)
xx[Claudia Frei-Wyssen (GLP, Uster), Beatrix Frey (FDP, Meilen) ]xx (1)
xx[Tobias Weidmann (SVP, Hettlingen), Marcel Suter (SVP, Thalwil), ]xx (61)
xx[Maria Rita Marty (SVP, Volketswil) ]xx (42)
xx[Hans-Peter Amrein (SVP, Küsnacht; fraktionslos) ]xx (1)
xx[Ratspräsident Benno Scherrer:]xx (239)
xx[Ratspräsident Benno Scherrer:]xx (189)
xx[Ratspräsident Benno Scherrer:]xx (315)
xx[Ratspräsident Benno Scherrer:]xx (314)
xx[Ratspräsident Benno Scherrer:]xx (215)
xx[Ratspräsident Benno Scherrer:]xx (294)
xx[Ratspräsident Benno Scherrer:]xx (338)
xx[Ratspräsident Benno Scherrer:]xx (224)
xx[Ratspräsident Benno Scherrer:]xx (197)
xx[Ratspräsident Benno Scherrer:]xx (265)
xx[Ratspräsident Benno Scherrer:]xx (232)
xx[Ratspräsident Benno Scherrer:]xx (280)
xx[Ratspräsident Benno Scherrer:]xx (241)
xx[Ratspräsident Benno Scherrer:]xx (280)
xx[Ratspr